In [15]:
import numpy as np
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

conclusion: 

Feature Selection: CCI, ADX, Chaikin A/D
Model Selection: best using SVC with poly kernel
Hyperparameter Tuning: d = 2 c = 0.3

In [16]:
'''
CONST
'''
FILEPATH_STOCK_PRICE = "./tr_eikon_eod_data.csv"
STOCK_NAMES = ['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GS.N']

In [17]:
''' 
UTILS
'''
def sequential_train_test_split(data, ratio = 0.8):
    df = pd.DataFrame(data.dropna()).to_numpy()
    split_index = int(len(df) * ratio)
    return df[:split_index], df[split_index:]


In [18]:
'''
BENCHMARK
'''

def benchmark_accuracy(prediction, test):
    prediction = pd.DataFrame(prediction)
    test = pd.DataFrame(test)

    return (prediction == test).sum() / len(prediction)


In [19]:
''' 
MODEL
'''

def predict_position(train, test):
    # Train Hidden Markov Model
    model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000)
    model.fit(train.reshape(-1, 1))

    # Predict hidden states for test data
    hidden_states = model.predict(test.reshape(-1, 1))

    # Define a function to map hidden states to rise or fall
    def map_to_rise_or_fall(state):
        return 1 if state == 0 else -1

    # Map hidden states to rise or fall
    predicted_rise_fall = np.array(list(map(map_to_rise_or_fall, hidden_states)))

    return predicted_rise_fall

In [27]:
filename = 'gs_output.csv'
stockname = 'GS.N'

apple_data = pd.read_csv(filename, index_col = 0, parse_dates = True)
apple_data['SMA_EMA_ratio'] = apple_data['SMA'] / apple_data['EMA']
apple_data['direction'] = np.where(apple_data[stockname] <= apple_data[stockname].shift(-1), 1, -1)

apple_data

,GS.N,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,direction
Date,,,,,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2010-04-01,173.08,9.7641,15.2302,50.3525,25.5682,-119.6258,100.0000,64.2857,-177810600.0,8.118063e+08,137.0128,135.9827,1.007575,1
2010-05-01,176.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2010-06-01,174.26,82.1737,80.0595,49.2538,31.4835,85.6424,71.4286,14.2857,-346418300.0,6.644775e+08,110.7708,112.1395,0.987795,1
2010-07-01,177.67,9.8352,26.4734,37.8543,33.0676,-128.0550,100.0000,71.4286,-430663700.0,6.215069e+08,107.1164,106.6629,1.004252,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,221.54,19.9620,20.9447,32.7234,22.7421,-152.1742,100.0000,42.8571,-240254198.0,6.437021e+08,201.4811,200.4351,1.005219,1
2018-06-26,221.58,17.8048,19.8747,28.5329,24.4674,-140.4348,100.0000,35.7143,-237561311.0,6.446095e+08,200.4011,199.2830,1.005611,-1
2018-06-27,220.18,14.1021,17.2896,28.0921,24.3184,-105.2860,92.8571,28.5714,-240664932.0,6.415525e+08,199.3360,198.2927,1.005261,1


In [21]:
scaler = MinMaxScaler()

# Normalize the selected columns
normalized_columns = scaler.fit_transform(apple_data.iloc[:, :-1])

# Create a new DataFrame with the normalized values
normalized_df = pd.DataFrame(normalized_columns, columns=apple_data.columns[:-1])

# Display the normalized DataFrame
apple_data.iloc[:, :-1] = normalized_df
apple_data

,AMZN.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,direction
Date,,,,,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2010-04-01,0.015407,0.547956,0.743189,0.642768,0.470929,0.506203,0.571429,0.857143,0.125422,0.082504,0.010625,0.010581,0.445655,1
2010-05-01,0.015888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2010-06-01,0.014402,0.745369,0.803009,0.359953,0.656789,0.511902,0.571429,0.142857,0.094772,0.081320,0.005648,0.005986,0.349605,-1
2010-07-01,0.013031,0.186826,0.158464,0.047981,0.663867,0.317693,0.857143,0.214286,0.048608,0.064585,0.003012,0.001564,0.810013,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.947041,0.253876,0.500732,0.476692,0.662018,0.191420,1.000000,0.857143,0.898056,0.983911,0.863689,0.859967,0.543386,1
2018-06-26,0.964063,0.221629,0.322018,0.353668,0.612456,0.308819,0.928571,0.785714,0.904173,0.986523,0.862580,0.855759,0.599412,-1
2018-06-27,0.945433,0.178958,0.216319,0.490156,0.553130,0.320307,0.857143,0.714286,0.897378,0.980226,0.862865,0.855806,0.603685,1


In [104]:
# apple_data = apple_data[['SlowK', 'RSI', 'CCI', 'Aroon Down', 'OBV', 'direction']]

In [8]:
train_data, test_data = sequential_train_test_split(apple_data)
X_train, y_train, X_test, y_test = train_data[:, :-1], train_data[:, -1], test_data[:, :-1], test_data[:, -1]

In [50]:
X_train

array([[0.01883667, 0.79708231, 0.85194069, ..., 0.02191803, 0.02147607,
        0.43614585],
       [0.01622904, 0.9071284 , 0.84424684, ..., 0.0337416 , 0.03448778,
        0.26904463],
       [0.01589451, 0.11149665, 0.1094004 , ..., 0.04589134, 0.04162706,
        0.95070081],
       ...,
       [0.53766059, 0.68329218, 0.72737065, ..., 0.4305807 , 0.43008523,
        0.43860904],
       [0.54156345, 0.72807436, 0.72102435, ..., 0.4332005 , 0.43240882,
        0.45046602],
       [0.5410831 , 0.67998392, 0.70044166, ..., 0.43543349, 0.43382752,
        0.48296742]])

In [9]:
from sklearn.svm import SVC
model = SVC(kernel='rbf', gamma=0.7, C=1.0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
benchmark_accuracy(pred, y_test)

0    0.494709
dtype: float64

In [10]:
best_acc = 0
best_g = -1
best_c = -1

selected_features = [4, 5, 9]

for g in range(1, 11, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='rbf', gamma=g / 10, C=c / 10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        print(f"acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_g = g
            best_c = c
        print(f"best acc: {best_acc}, d: {best_g}, c: {best_c}")
    
print(f"best acc: {best_acc}, g: {best_g}, c: {best_c}")

acc: 0.529 up: 1.0
best acc: 0.5291005291005291, d: 1, c: 1
acc: 0.529 up: 1.0
best acc: 0.5291005291005291, d: 1, c: 1
acc: 0.529 up: 1.0
best acc: 0.5291005291005291, d: 1, c: 1
acc: 0.529 up: 1.0
best acc: 0.5291005291005291, d: 1, c: 1
acc: 0.529 up: 1.0
best acc: 0.5291005291005291, d: 1, c: 1
acc: 0.529 up: 1.0
best acc: 0.5291005291005291, d: 1, c: 1
acc: 0.529 up: 1.0
best acc: 0.5291005291005291, d: 1, c: 1
acc: 0.529 up: 1.0
best acc: 0.5291005291005291, d: 1, c: 1
acc: 0.529 up: 1.0
best acc: 0.5291005291005291, d: 1, c: 1
acc: 0.534 up: 0.989
best acc: 0.5343915343915344, d: 1, c: 10
acc: 0.529 up: 0.979
best acc: 0.5343915343915344, d: 1, c: 10
acc: 0.532 up: 0.96
best acc: 0.5343915343915344, d: 1, c: 10
acc: 0.521 up: 0.939
best acc: 0.5343915343915344, d: 1, c: 10
acc: 0.516 up: 0.902
best acc: 0.5343915343915344, d: 1, c: 10
acc: 0.508 up: 0.894
best acc: 0.5343915343915344, d: 1, c: 10
acc: 0.508 up: 0.894
best acc: 0.5343915343915344, d: 1, c: 10
acc: 0.503 up: 0.873

KeyboardInterrupt: 

In [151]:
best_acc = 0
best_c = -1

selected_features = [4, 5, 9]

for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='sigmoid', C= c/10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    print(f"acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_g = g
        best_c = c
    print(f"best acc: {best_acc}, c: {best_c}")
    
print(f"best acc: {best_acc}, c: {best_c}")

acc: 0.519 up: 0.0
best acc: 0.5185185185185185, c: 1
acc: 0.519 up: 0.0
best acc: 0.5185185185185185, c: 1
acc: 0.521 up: 0.198
best acc: 0.5211640211640212, c: 3
acc: 0.532 up: 0.098
best acc: 0.5317460317460317, c: 4
acc: 0.542 up: 0.743
best acc: 0.5423280423280423, c: 5
acc: 0.537 up: 0.749
best acc: 0.5423280423280423, c: 5
acc: 0.503 up: 0.037
best acc: 0.5423280423280423, c: 5
acc: 0.489 up: 0.071
best acc: 0.5423280423280423, c: 5
acc: 0.495 up: 0.082
best acc: 0.5423280423280423, c: 5
acc: 0.487 up: 0.101
best acc: 0.5423280423280423, c: 5
acc: 0.484 up: 0.108
best acc: 0.5423280423280423, c: 5
acc: 0.497 up: 0.132
best acc: 0.5423280423280423, c: 5
acc: 0.481 up: 0.148
best acc: 0.5423280423280423, c: 5
acc: 0.481 up: 0.159
best acc: 0.5423280423280423, c: 5
acc: 0.468 up: 0.172
best acc: 0.5423280423280423, c: 5
acc: 0.46 up: 0.18
best acc: 0.5423280423280423, c: 5
acc: 0.463 up: 0.188
best acc: 0.5423280423280423, c: 5
acc: 0.463 up: 0.204
best acc: 0.5423280423280423, c: 

In [14]:
best_acc = 0
best_d = -1
best_c = -1

selected_features = [4, 5, 9]

for d in range(1, 5, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree=d, C=c/10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        print(f"acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_d = d
            best_c = c
        print(f"best acc: {best_acc}, d: {best_d}, c: {best_c}")
    
print(f"best acc: {best_acc}, d: {best_d}, c: {best_c}")

acc: 0.468 up: 0.743
best acc: 0.46825396825396826, d: 1, c: 1
acc: 0.476 up: 0.73
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.468 up: 0.712
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.471 up: 0.714
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.471 up: 0.714
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.471 up: 0.714
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.471 up: 0.714
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.471 up: 0.714
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.471 up: 0.693
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.471 up: 0.698
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.471 up: 0.693
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.471 up: 0.693
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.471 up: 0.693
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.474 up: 0.696
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.471 up: 0.693
best acc: 0.47619047619047616, d: 1, c: 2
acc: 0.471 up: 0.693
best acc: 0.47619047619047616, d: 1

In [54]:
best_acc = 0
best_c = -1

selected_features = [4, 5, 9]

# for d in range(1, 5, 1):
for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='poly', degree=2, C = c / 10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    print(f"acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    print(f"best acc: {best_acc}, c: {best_c}")
    
print(f"best acc: {best_acc}, c: {best_c}")

acc: 0.619 up: 0.28
best acc: 0.6190476190476191, c: 1
acc: 0.611 up: 0.257
best acc: 0.6190476190476191, c: 1
acc: 0.622 up: 0.246
best acc: 0.6216931216931217, c: 3
acc: 0.606 up: 0.235
best acc: 0.6216931216931217, c: 3
acc: 0.608 up: 0.238
best acc: 0.6216931216931217, c: 3
acc: 0.606 up: 0.235
best acc: 0.6216931216931217, c: 3
acc: 0.606 up: 0.235
best acc: 0.6216931216931217, c: 3
acc: 0.608 up: 0.233
best acc: 0.6216931216931217, c: 3
acc: 0.608 up: 0.233
best acc: 0.6216931216931217, c: 3
acc: 0.608 up: 0.228
best acc: 0.6216931216931217, c: 3
acc: 0.608 up: 0.228
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3
acc: 0.611 up: 0.23
best acc: 0.6216931216931217, c: 3


In [ ]:
# Feature Engineering

# Feature Selection

In [43]:
X = apple_data.dropna().iloc[:, :-1]
y = apple_data.dropna().iloc[:, -1]

In [44]:
X

,AAPL.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio
Date,,,,,,,,,,,,,
2010-04-01,0.018837,0.797082,0.851941,0.769280,0.413634,0.658869,0.428571,1.000000,0.466485,0.197673,0.021918,0.021476,0.436146
2010-06-01,0.016229,0.907128,0.844247,0.536156,0.374250,0.748379,0.571429,1.000000,0.417465,0.386577,0.033742,0.034488,0.269045
2010-07-01,0.015895,0.111497,0.109400,0.287327,0.221484,0.249224,1.000000,0.428571,0.311572,0.333500,0.045891,0.041627,0.950701
2010-11-01,0.015491,0.258310,0.233819,0.445504,0.149730,0.329648,0.000000,0.285714,0.566643,0.506703,0.070674,0.068125,0.680743
2010-12-01,0.013441,0.723159,0.787620,0.558717,0.137354,0.660432,0.357143,0.000000,0.561133,0.530421,0.072551,0.072439,0.398836
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.929088,0.244971,0.261688,0.215016,0.272481,0.283454,1.000000,0.142857,0.572554,0.836229,0.812646,0.803231,0.652194
2018-06-26,0.942658,0.308398,0.263699,0.201690,0.271435,0.383573,0.928571,0.071429,0.576126,0.836040,0.808467,0.799351,0.645859
2018-06-27,0.941037,0.375016,0.287289,0.339535,0.260639,0.428531,0.857143,0.000000,0.572450,0.832650,0.804889,0.798193,0.587664


In [45]:
y

Date
2010-04-01    1
2010-06-01   -1
2010-07-01    1
2010-11-01   -1
2010-12-01    1
             ..
2018-06-25    1
2018-06-26   -1
2018-06-27    1
2018-06-28   -1
2018-06-29   -1
Name: direction, Length: 1887, dtype: int64

In [46]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Assuming 'X' contains the features and 'y' contains the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Selecting the most relevant features using mutual information
selector = SelectKBest(score_func=mutual_info_classif, k=5)  # Select top 5 features
X_train_selected = selector.fit_transform(X_train, y_train)
selected_features = X.columns[selector.get_support()]

# Train a model using the selected features
model = LogisticRegression()
model.fit(X_train_selected, y_train)

# Evaluate the model on the test set
X_test_selected = selector.transform(X_test)
accuracy = model.score(X_test_selected, y_test)

# Display the most relevant features and model accuracy
print("Selected features:", selected_features)
print("Model accuracy:", accuracy)

Selected features: Index(['SlowK', 'RSI', 'CCI', 'OBV', 'EMA'], dtype='object')
Model accuracy: 0.5238095238095238


In [29]:
best_acc = 0

for i in range(len(apple_data.columns) - 1):
    curr_X_train = X_train[:, 1].reshape(-1, 1)
    curr_X_test = X_test[:, i].reshape(-1, 1)
    model = SVC(kernel='rbf', gamma= 1.0, C= 1.0)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

feature 0: acc: 0.476 up: 0.545
feature 1: acc: 0.471 up: 0.397
feature 2: acc: 0.492 up: 0.37
feature 3: acc: 0.437 up: 0.5
feature 4: acc: 0.487 up: 0.81
feature 5: acc: 0.466 up: 0.423
feature 6: acc: 0.5 up: 0.669
feature 7: acc: 0.476 up: 0.365
feature 8: acc: 0.471 up: 0.011
feature 9: acc: 0.471 up: 0.0
feature 10: acc: 0.497 up: 0.635
feature 11: acc: 0.495 up: 0.643
feature 12: acc: 0.492 up: 0.635


In [13]:
import itertools

all_features = list(range(len(apple_data.columns) - 1))
selected_features = list(itertools.permutations(all_features, 2))

best_acc = 0
best_selected_features = []

for i in range(1, len(apple_data.columns) - 1):
    possible_selected_features = list(itertools.permutations(all_features, i))
    for selected_features in possible_selected_features:
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}, features: {selected_features}")

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
        print(f"best acc: {best_acc} features: {best_selected_features}")

feature 1: acc: 0.529 up: 1.0, features: (0,)
best acc: 0.5291005291005291 features: (0,)
feature 1: acc: 0.492 up: 0.571, features: (1,)
best acc: 0.5291005291005291 features: (0,)
feature 1: acc: 0.497 up: 0.587, features: (2,)
best acc: 0.5291005291005291 features: (0,)
feature 1: acc: 0.511 up: 0.738, features: (3,)
best acc: 0.5291005291005291 features: (0,)
feature 1: acc: 0.503 up: 0.878, features: (4,)
best acc: 0.5291005291005291 features: (0,)
feature 1: acc: 0.481 up: 0.571, features: (5,)
best acc: 0.5291005291005291 features: (0,)
feature 1: acc: 0.489 up: 0.251, features: (6,)
best acc: 0.5291005291005291 features: (0,)
feature 1: acc: 0.529 up: 1.0, features: (7,)
best acc: 0.5291005291005291 features: (0,)
feature 1: acc: 0.529 up: 1.0, features: (8,)
best acc: 0.5291005291005291 features: (0,)
feature 1: acc: 0.529 up: 1.0, features: (9,)
best acc: 0.5291005291005291 features: (0,)
feature 1: acc: 0.526 up: 0.992, features: (10,)
best acc: 0.5291005291005291 features: 

KeyboardInterrupt: 

array([[0.01883667, 0.79708231],
       [0.01622904, 0.9071284 ],
       [0.01589451, 0.11149665],
       ...,
       [0.53766059, 0.68329218],
       [0.54156345, 0.72807436],
       [0.5410831 , 0.67998392]])